Exam requests: 

1. Write a class PostcardList that satisfies the tests (defined using unittest). 

SPECIFICATIONS: 
Class PostcardList reads/writes Postcard messages from/to a properly formatted file: each record (i.e., file's line) is a Postcard. 
The Postcard format is "date:$(DATE); from:$(SENDER); to:$(RECEIVER);" (e.g., "date:2009-12-24; from:Daisy; to:Donald;"). For the sake of simplicity, the message is not considered.

Class PostcardList must have at least these attributes:
 - _file: file name, eventually with the full path.
 - _postcards: list of postcards read from _file.
 - _date: is a dict where the key is the string date, and the value is a list of indices. Each index refers to the corresponding record.
 - _from: is a dict where the key is the string sender, and the value is a list of indices. Each index refers to the corresponding record.
 - _to:   is a dict where the key is the string receiver, and the value is a list of indices. Each index refers to the corresponding record.

Class PostcardList must manage the I/O file through the following member functions. Note that you are free to choose the appropriate arguments, but the function names must be as follows:  
 - writeFile(self,...): write self.{_date,_from,_to} to self._file  
 - readFile(self,...): from self._file read self.{_date,_from,_to}    
 - parsePostcards(self,...): parse self._postcards, set self.{_date,_from,_to} 
 - updateFile(self,...): as write but appending to self._file 
 - updateLists(self,...): as read but appending to self._postcards 
 - getNumberOfPostcards(self): returns length of self._postcards

Class PostcardList must manage the sorting of dates/senders/receivers. Note that the names and arguments of the functions are fixed as follows: 
 - getPostcardsByDateRange(self,date_range): returns the postcards within a date_range
 - getPostcardsBySender(self, sender): returns the postcards from a sender
 - getPostcardsByReceiver(self, receiver): returns the postcards to a receiver

In [8]:
import unittest
import datetime # use this module to deal with dates:  https://docs.python.org/3/library/datetime.html
from collections import defaultdict

class PostcardList: 
    def __init__(self):
        self._file = list() #keep track of multiple file
        self._postcards = list()
        self._date= {}   #   defaultdict(list) #dict(str(date),list(indexes where data is))
        self._from={}
        self._to={}
        self._size=0
#date are stored as string, because we are required to do so, otherwise it's even
 #possible to store them directly as dates
        
        
    def getNumberOfPostcards(self):
        return self._size
    
    def __len__(self):
        return self._size
    
    def parsePostcards(self,input_list):
        for i in input_list:
                i=i.split("; ")
                i[0]=i[0].replace('date:','') #date: is deleted
                i[1]=i[1].replace("from:","")
                i[2]=i[2].replace("to:","")
                if i[0] not in self._date:
                    self._date[i[0]] = []
                self._date[i[0]].append(self._size)
                if i[1] not in self._from:
                    self._from[i[1]] = []
                self._from[i[1]].append(self._size)
                if i[2] not in self._to:
                    self._to[i[2]] = []
                self._to[i[2]].append(self._size)
                    
                #self._date[i[0]].append(self._size)
                #self._from[i[1]].append(self._size)
                #self._to[i[2]].append(self._size)
                self._size+=1
    
    
    def readFile(self,filename):
        ##delete everything
        self._file.append(filename)
        with open(filename,'r') as f:
            for i in f:
                i=i[:-1]
                self._postcards.append(i)
        self.parsePostcards(self._postcards)
    
    def writeFile(self,filename):
        with open(filename,'w') as f:
            for i in self._postcards:
                print(i,file=f)
    
    #updateLists(self,...): as read but appending to self._postcards
    def updateLists(self,newfilename):
        self._file.append(newfilename)
        with open(newfilename,'r') as f:
            tmp =list()
            for i in f:
                i=i[:-2]
                self._postcards.append(i)
                tmp.append(i) #list with all the newcontent
        self.parsePostcards(tmp)  
        
    def updateFile(self,filename):
        with open(filename,'a+') as f:
            for i in self._postcards:
                print(i,file=f)
                
                
                
    #range methods            
                
    def getPostcardsByDateRange(self,date_range):  #data range assumed to be a list of strings
        l=list()
        date_range=tuple(date_range)               #become const as tuple
        l_bound=datetime.datetime.strptime(date_range[0], "%Y-%m-%d")
        u_bound=datetime.datetime.strptime(date_range[1], "%Y-%m-%d")          
    #    u_bound=datetime.date.fromisoformat(date_string[1])
        idx=0
        for i in self._date:
            tmp=datetime.datetime.strptime(i, "%Y-%m-%d")
            if (l_bound<=tmp<=u_bound):
                l.append(   (self._postcards[idx])   )
            idx+=1
        return l

In [9]:
p=PostcardList()
p.readFile("exam_postcard_list0.txt")
test=p.getPostcardsByDateRange(["2008-01-01","2009-01-01"])
test
#date.fromisoformat(date_string)

FileNotFoundError: [Errno 2] No such file or directory: 'exam_postcard_list0.txt'

In [10]:
########################
# TO COMMENT 
#from exam_solution import PostcardList # overrides the class define above
########################


class Test(unittest.TestCase):

    @classmethod
    def setUpClass(self):
        self._files = ['./exam_postcard_list{}.txt'.format(i) \
                           for i in range(10)]
        self._PstLst = [PostcardList() for f in self._files]
        [p.readFile(f) for f,p in zip(self._files,self._PstLst)]

    def test_missing_attributes(self): 
        attributes_to_define = ['_file','_postcards','_date','_from','_to',\
                                'writeFile','readFile','parsePostcards',\
                                'updateFile','updateLists',\
                                'getNumberOfPostcards',\
                                'getPostcardsByDateRange',\
                                'getPostcardsBySender',\
                                'getPostcardsByReceiver']
        for attr in attributes_to_define:
            if attr not in dir(self._PstLst[0]): 
                raise Exception(attr+' is missing')

    def test_check_getPostcardByDateRange(self): 
        dr_test = []
        for i in [0,1,4,6]: 
            dr_test.append(\
                self._PstLst[i].getPostcardsByDateRange(date_range=\
                   (datetime.datetime.strptime('2008-1-1', "%Y-%m-%d"),\
                    datetime.datetime.strptime('2010-12-31', "%Y-%m-%d"))))
        self.assertListEqual(sorted(dr_test[0]),sorted(['date:2010-06-23; from:Sneezy; to:Alice;\n', 'date:2009-12-12; from:Dopey; to:Peter;\n', 'date:2008-03-23; from:Sneezy; to:Pluto;\n', 'date:2008-06-03; from:Goofy; to:Pluto;\n'])) 
        self.assertEqual(len(dr_test[1]),1)
        self.assertListEqual(sorted(dr_test[2]),sorted(['date:2010-03-30; from:Louie; to:Sneezy;\n', 'date:2010-03-05; from:Goofy; to:Dopey;\n', 'date:2009-11-08; from:Daisy; to:Minnie;\n', 'date:2010-07-13; from:Bashful; to:Louie;\n', 'date:2008-06-29; from:Huey; to:Dopey;\n', 'date:2009-01-04; from:Alice; to:Hook;\n', 'date:2009-04-28; from:Bashful; to:Mickey;\n']))
        self.assertEqual(len(dr_test[3]),7)

    def test_check_getPostcardBySender(self): 
        s_test = []
        for i in [2,3,5,7]:
            for sender in ['Daisy','Bashful','Peter']:
                s_test.append(\
                    self._PstLst[i].getPostcardsBySender(sender=sender))
        self.assertIn('date:2011-12-11; from:Daisy; to:Goofy;\n',s_test[0])
        self.assertListEqual(s_test[1],[])
        self.assertEqual(len(s_test[2]),1)
        self.assertEqual(len(s_test[5]),0)
        self.assertGreater(len(s_test[8]),len(s_test[9]))

    def test_check_getPostcardByReceiver(self): 
        r_test = []
        for i in [0,3,8,9]:
            for receiver in ['Peter','Hook','Alice','SnowWhite']:
                r_test.append(\
                  self._PstLst[i].getPostcardsByReceiver(receiver=receiver))
        self.assertIn('date:2009-12-12; from:Dopey; to:Peter;\n',r_test[0])
        self.assertListEqual(r_test[1],['date:2016-10-23; from:Sneezy; to:Hook;\n'])
        self.assertEqual(len(r_test[2]),2)
        self.assertEqual(len(r_test[6]),3)

    def test_check_sent_received_when(self): 
        srw_test = []
        for ii in range(10):
            for jj in ["Mickey","Minnie","Goofy","Pluto","Donald","Daisy","$crooge","Huey","Dewey","Louie","Peter","Hook","Alice","SnowWhite","Doc","Sleepy","Sneezy","Bashful","Dopey","Happy","Grumpy"]:
                try:
                    srw_test.append(self._PstLst[ii]._from[jj])
                    srw_test.append(self._PstLst[ii]._to[jj])
                    srw_test.append(self._PstLst[ii]._date[jj])
                except: pass
        self.assertListEqual(srw_test[0],[1])
        self.assertListEqual(srw_test[130],[14,25])
        self.assertListEqual(srw_test[138],[10])
        self.assertListEqual(srw_test[140],[15])
        self.assertListEqual(srw_test[192],[13,26])
        self.assertListEqual(srw_test[203],[6, 9, 11, 12, 24, 31, 42])

if __name__ == '__main__':

    unittest.main()

E
ERROR: /run/user/1000/jupyter/kernel-836e0e13-5a03-43fc-a1f1-fbc5a2109be5 (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/run/user/1000/jupyter/kernel-836e0e13-5a03-43fc-a1f1-fbc5a2109be5'

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)


SystemExit: True

/home/w-wilson/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [38]:
d1=datetime.datetime.strptime("2010-06-04", "%Y-%m-%d")
d2=datetime.datetime.strptime("2010-01-01", "%Y-%m-%d")
d2<d1
d3=datetime.datetime.strptime("2010-06-04", "%Y-%m-%d")


True

In [135]:
??datetime.date

In [30]:
dt = datetime.datetime.strptime("2012-01-01", "%y-%m-%d")

ValueError: time data '2012-01-01' does not match format '%y-%m-%d'